# Сравнение производительности OLAP систем

Для чистого сравнения производительности одинаковые бд + таблица для Clickhouse и Vertica

In [1]:

import pandas as pd

# Clickhouse

In [2]:
%%time

df = pd.read_csv('/Users/elinachertova/PycharmProjects/UGC_service/ugc/storage_research/views_data.csv')

CPU times: user 17.8 s, sys: 3.69 s, total: 21.5 s
Wall time: 23.1 s


In [3]:
%%time
filename = '/Users/elinachertova/PycharmProjects/UGC_service/ugc/storage_research/views_data.csv'
file = open(filename, 'r').read()

CPU times: user 420 ms, sys: 1.56 s, total: 1.98 s
Wall time: 2.99 s


'/Users/elinachertova/PycharmProjects/UGC_service/ugc/storage_research/views_data.csv'

In [4]:
df.shape

(10000000, 7)

In [5]:
df.columns

Index(['Unnamed: 0', 'id', 'user_id', 'movie_id', 'views_time', 'movie_time',
       'event_time'],
      dtype='object')

In [6]:
df.head(2)


,Unnamed: 0,id,user_id,movie_id,views_time,movie_time,event_time
0,0,0,44699f18-e1f2-4ecf-bd79-b5b34896757d,80819ce6-8266-425f-9e40-ac48732aeefc,15280,6345,2022-12-23 22:56:37.646839
1,1,1,7a169ccd-1885-48d0-90c6-a0fda6e4a1de,0aa12756-52a8-4e1a-9c45-a23a2f09f27f,15458,24974,2022-12-23 22:56:37.657832


In [7]:
df = df[['id', 'user_id', 'movie_id', 'views_time', 'movie_time', 'event_time']]
# [:5000000]

In [99]:
from clickhouse_driver import Client

client = Client(host='localhost', settings={"use_numpy": True})

In [54]:
# client.execute('DROP TABLE test.views;')
# client.execute('DROP DATABASE test;')

In [56]:
client.execute('CREATE DATABASE IF NOT EXISTS test ON CLUSTER company_cluster')



[array(['clickhouse-node1', 9000, 0, '', 3, 0], dtype=object),
 array(['clickhouse-node3', 9000, 0, '', 2, 0], dtype=object),
 array(['clickhouse-node4', 9000, 0, '', 1, 0], dtype=object),
 array(['clickhouse-node2', 9000, 0, '', 0, 0], dtype=object)]

In [57]:
client.execute('CREATE TABLE test.views ON CLUSTER company_cluster (id Int64, user_id UUID, movie_id UUID, views_time UInt16, movie_time UInt16, event_time DateTime) Engine=MergeTree() ORDER BY id')



[array(['clickhouse-node1', 9000, 0, '', 3, 0], dtype=object),
 array(['clickhouse-node3', 9000, 0, '', 2, 0], dtype=object),
 array(['clickhouse-node2', 9000, 0, '', 1, 0], dtype=object),
 array(['clickhouse-node4', 9000, 0, '', 0, 0], dtype=object)]

## Вставка таблицы 

### Вставка пачкой по 5000

In [97]:
query = 'INSERT INTO test.views VALUES'
step = 5000

In [59]:
%%time
for i in range(0, df.shape[0], step):
    batch = df[i:i+step].copy()
    client.insert_dataframe(query, batch)

CPU times: user 56 s, sys: 2.83 s, total: 58.8 s
Wall time: 1min 37s


### Вставка 1000 строк пачкой по 3

In [100]:
%%time
step = 3
for i in range(0, 1000, step):
    batch = df[i:i+step].copy()
    client.insert_dataframe(query, batch)

CPU times: user 1.1 s, sys: 160 ms, total: 1.26 s
Wall time: 4.35 s


In [12]:
df.head()

,id,user_id,movie_id,views_time,movie_time,event_time
0,0,44699f18-e1f2-4ecf-bd79-b5b34896757d,80819ce6-8266-425f-9e40-ac48732aeefc,15280,6345,2022-12-23 22:56:37.646839
1,1,7a169ccd-1885-48d0-90c6-a0fda6e4a1de,0aa12756-52a8-4e1a-9c45-a23a2f09f27f,15458,24974,2022-12-23 22:56:37.657832
2,2,f3ee00c2-5ab4-4388-947e-9003340ada16,36153728-96cd-4a6c-a085-24cf81f1a747,867,35371,2022-12-23 22:56:37.657912
3,3,af16cedc-aa9b-4f41-b2b9-35b45bd031a7,04714c8e-f720-4dc6-92a8-66e9068f04f9,22079,1687,2022-12-23 22:56:37.657915
4,4,647a3948-d905-4356-8aa5-a2a24f878547,73ab4320-7c07-4058-b08a-abb6dfda9081,9868,20293,2022-12-23 22:56:37.657917


In [65]:
df.tail()

,id,user_id,movie_id,views_time,movie_time,event_time
9999995,9999995,047e43b0-385f-47cb-82c2-c29f9a52509d,b5aa31ce-5990-496a-9142-f2beeb354eb6,24774,3559,2022-12-23 22:56:49.083412
9999996,9999996,088b3bac-0ec7-4ad8-95e1-4cb64cf1f382,bbba2af4-6b09-4802-bee3-d1b33ee77d8e,26056,26361,2022-12-23 22:56:49.083413
9999997,9999997,347a3132-ffeb-485d-9672-22b890854f2b,b601b430-f4c2-4a45-9618-05d02b1e9141,22003,2487,2022-12-23 22:56:49.083414
9999998,9999998,82d4b620-3caf-46df-b1e1-7fb2e0a88255,c041e35a-ccf9-47ff-ad7c-d6b8bcb659b9,24279,19364,2022-12-23 22:56:49.083415
9999999,9999999,45c97ad0-1b07-4379-af96-58d4fb904850,4e8e769e-59dc-4d88-a502-381f3a34b29f,9145,32280,2022-12-23 22:56:49.083416


## Чтение таблицы 

### Вся таблица 

In [61]:
%%time
data = client.execute('SELECT * FROM test.views LIMIT 10000000')

CPU times: user 48.6 s, sys: 4.28 s, total: 52.9 s
Wall time: 57.1 s


In [101]:
%%time
data = client.execute('SELECT * FROM test.views LIMIT 1554500')

CPU times: user 12.3 s, sys: 5.26 s, total: 17.6 s
Wall time: 22.3 s


### SELECT + WHERE

In [64]:
%%time
client.execute("SELECT * FROM test.views WHERE user_id='647a3948-d905-4356-8aa5-a2a24f878547'")

CPU times: user 7.78 ms, sys: 29.1 ms, total: 36.8 ms
Wall time: 1.46 s


[array([4, UUID('647a3948-d905-4356-8aa5-a2a24f878547'),
        UUID('73ab4320-7c07-4058-b08a-abb6dfda9081'), 9868, 20293,
        datetime.datetime(2022, 12, 23, 22, 56, 37)], dtype=object)]

In [66]:
%%time
client.execute("SELECT * FROM test.views WHERE user_id='088b3bac-0ec7-4ad8-95e1-4cb64cf1f382'")

CPU times: user 5.8 ms, sys: 12 ms, total: 17.8 ms
Wall time: 413 ms


[array([9999996, UUID('088b3bac-0ec7-4ad8-95e1-4cb64cf1f382'),
        UUID('bbba2af4-6b09-4802-bee3-d1b33ee77d8e'), 26056, 26361,
        datetime.datetime(2022, 12, 23, 22, 56, 49)], dtype=object)]

In [67]:
%%time
client.execute("SELECT * FROM test.views WHERE movie_id='c041e35a-ccf9-47ff-ad7c-d6b8bcb659b9'")

CPU times: user 5.91 ms, sys: 10.3 ms, total: 16.2 ms
Wall time: 775 ms


[array([9999998, UUID('82d4b620-3caf-46df-b1e1-7fb2e0a88255'),
        UUID('c041e35a-ccf9-47ff-ad7c-d6b8bcb659b9'), 24279, 19364,
        datetime.datetime(2022, 12, 23, 22, 56, 49)], dtype=object)]

### Real-time 

Часть кода (файл check_insert) выполняет вставку, пока данный тест проверяет чтение.

In [68]:
%%time
client.execute("SELECT * FROM test.views WHERE user_id='647a3948-d905-4356-8aa5-a2a24f878547'")

CPU times: user 7.3 ms, sys: 22.5 ms, total: 29.8 ms
Wall time: 531 ms


[array([4, UUID('647a3948-d905-4356-8aa5-a2a24f878547'),
        UUID('73ab4320-7c07-4058-b08a-abb6dfda9081'), 9868, 20293,
        datetime.datetime(2022, 12, 23, 22, 56, 37)], dtype=object),
 array([4, UUID('647a3948-d905-4356-8aa5-a2a24f878547'),
        UUID('73ab4320-7c07-4058-b08a-abb6dfda9081'), 9868, 20293,
        datetime.datetime(2022, 12, 23, 22, 56, 37)], dtype=object)]

# Vertica

In [102]:
import vertica_python

connection_info = {
    'host': '127.0.0.1',
    'port': 5433,
    'user': 'dbadmin',
    'password': '',
    'database': 'docker',
    'autocommit': True,
}

In [111]:
connection = vertica_python.connect(**connection_info)
cursor = connection.cursor()

In [112]:

cursor.execute("""
    CREATE TABLE views (
        id INTEGER NOT NULL,
        user_id UUID NOT NULL,
        movie_id UUID NOT NULL,
        views_time INTEGER NOT NULL,
        movie_time INTEGER NOT NULL,
        event_time Datetime NOT NULL
    );""")

In [113]:

list_of_tuples = df[['user_id', 'movie_id', 'views_time', 'movie_time', 'event_time']].to_records().tolist()

In [114]:
df.head()

,id,user_id,movie_id,views_time,movie_time,event_time
0,0,44699f18-e1f2-4ecf-bd79-b5b34896757d,80819ce6-8266-425f-9e40-ac48732aeefc,15280,6345,2022-12-23 22:56:37.646839
1,1,7a169ccd-1885-48d0-90c6-a0fda6e4a1de,0aa12756-52a8-4e1a-9c45-a23a2f09f27f,15458,24974,2022-12-23 22:56:37.657832
2,2,f3ee00c2-5ab4-4388-947e-9003340ada16,36153728-96cd-4a6c-a085-24cf81f1a747,867,35371,2022-12-23 22:56:37.657912
3,3,af16cedc-aa9b-4f41-b2b9-35b45bd031a7,04714c8e-f720-4dc6-92a8-66e9068f04f9,22079,1687,2022-12-23 22:56:37.657915
4,4,647a3948-d905-4356-8aa5-a2a24f878547,73ab4320-7c07-4058-b08a-abb6dfda9081,9868,20293,2022-12-23 22:56:37.657917


In [115]:
list_of_tuples = list_of_tuples[:1000]

## Вставка  

### 1000 строк по 5 шагов 

In [116]:
%%time
step = 5
for i in range(0, len(list_of_tuples), step):
    batch = list_of_tuples[i:i+step].copy()
    cursor.executemany('INSERT INTO views (id, user_id, movie_id, views_time, movie_time, event_time) VALUES '
                       '(?, ?, ?, ?, ?, ?)',
                       batch, use_prepared_statements=True)


CPU times: user 172 ms, sys: 53.4 ms, total: 226 ms
Wall time: 42.7 s


### 1000 строк по 3 шага 

In [117]:
%%time
step = 3
for i in range(0, len(list_of_tuples), step): 
    batch = list_of_tuples[i:i+step].copy()
    cursor.executemany('INSERT INTO views (id, user_id, movie_id, views_time, movie_time, event_time) VALUES '
                       '(?, ?, ?, ?, ?, ?)',
                       batch, use_prepared_statements=True)

CPU times: user 223 ms, sys: 57.3 ms, total: 280 ms
Wall time: 37.4 s


### 1000 строк по 1 шагу

In [118]:
%%time

for i in range(len(list_of_tuples)):
    cursor.execute(f"INSERT INTO views (id, user_id, movie_id, views_time, movie_time, event_time) VALUES ({list_of_tuples[i][0]}, '{list_of_tuples[i][1]}', '{list_of_tuples[i][2]}', {list_of_tuples[i][3]}, {list_of_tuples[i][4]}, '{list_of_tuples[i][5]}')")



CPU times: user 424 ms, sys: 89.5 ms, total: 513 ms
Wall time: 38.4 s


### Вставка всего файла 

In [119]:
%%time

cursor.execute(f"COPY views FROM LOCAL '{filename}' PARSER fcsvparser();")

CPU times: user 477 ms, sys: 736 ms, total: 1.21 s
Wall time: 2min 33s


## Чтение

### Чтение 1500000 строк

In [124]:
%%time

for _ in range(500):
    data = cursor.execute("""SELECT * FROM views;""").fetchall()


CPU times: user 47.8 s, sys: 3.39 s, total: 51.2 s
Wall time: 1min 4s


In [126]:
%%time
data  = cursor.execute("""SELECT * FROM views WHERE user_id='647a3948-d905-4356-8aa5-a2a24f878547';""").fetchall()


CPU times: user 2.47 ms, sys: 2.09 ms, total: 4.55 ms
Wall time: 33.7 ms


## Результат 

Далее проверять нет смысла, поскольку при вставке в 10млн строк Clickhouse отрабатывает за 57.1 секунду. На таких же данных Vertica пачками вставляет дольше 15мин (падает в какой-то момент). При вставке 1000 строк по 3 шага Vertica отрабатывает за 37.4 s секунды, Clickhouse за 4.35 s -> Clickhouse лучше подходит для вставки данных.

При чтении 1.5 млн строк Vertica отрабатывает за 1min 4s, Clickhouse за 22.3 s.
При поиске определенной записи Vertica работает лучше Clickhouse - 413 ms, Vertica - 33.7 ms.
Однако это единственный параметр, по которому Vertica лидирует -> выбираем Clickhouse.